In [34]:
from lifelines import CoxPHFitter
import pandas as pd
import numpy as np
from lifelines.datasets import load_regression_dataset

# Create the dataset
data = {
    "Follow_up_time": [12, 8, 15, 2, 6, 11],
    "Event_indicator": [1, 0, 1, 0, 1, 0],
    "Embedding_1": [0.25, 0.000000000000000000002, 0.000000000000000001, 0.25, 0.13, 0.98],
    "Embedding_2": [1.5, 0.0077, 0.1, 0.22, 0.00123, 0.68],
    "Embedding_3": [0.00005, 0.001, 0.15, 0.12, 0.0081, 0.81],
}

# Convert to a DataFrame
df = pd.DataFrame(data)


# Fit the Cox Proportional Hazards model
cph = CoxPHFitter()
cph.fit(df, duration_col="Follow_up_time", event_col="Event_indicator")
 

c:\Users\ASUS\pprojects\base\Lib\site-packages\lifelines\fitters\coxph_fitter.py:1614: ConvergenceWarning: Newton-Raphson failed to converge sufficiently. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-model
  warnings.warn(


<lifelines.CoxPHFitter: fitted with 6 total observations, 3 right-censored observations>

In [38]:
# Print the summary of the fitted model
print("Model Summary:")
cph.summary

Model Summary:


,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,cmp to,z,p,-log2(p)
covariate,,,,,,,,,,,
Embedding_1,5.369631,214.783518,9.893112,-14.020513,24.759774,8.146456e-07,5.662826e+10,0.0,0.542765,0.587292,0.767850
Embedding_2,-1.193602,0.303127,2.504996,-6.103304,3.716099,2.235470e-03,4.110376e+01,0.0,-0.476489,0.633726,0.658068
Embedding_3,-6.135554,0.002165,9.799668,-25.342550,13.071441,9.859856e-12,4.751763e+05,0.0,-0.626098,0.531251,0.912536


In [44]:
# Predict the hazard for each individual
df["Predicted_hazard"] = -cph.predict_partial_hazard(df)
print("\nPredicted Hazards:")
df[["Follow_up_time", "Event_indicator", "Predicted_hazard"]]


Predicted Hazards:


,Follow_up_time,Event_indicator,Predicted_hazard
0,12,1,-0.758596
1,8,0,-1.169633
2,15,1,-0.419926
3,2,0,-1.674584
4,6,1,-2.268008
5,11,0,-0.706668


In [43]:
# Plot the survival function for the individuals
print("\nSurvival Predictions:")
for i in range(len(df)):
    survival_function = cph.predict_survival_function(df.iloc[[i]])
    print(f"Individual {i + 1}:")
    print(survival_function.head())

# something like S(12.0)=0.455572: 45.6% chance of surviving beyond time 12.


Survival Predictions:
Individual 1:
             0
2.0   1.000000
6.0   0.867172
8.0   0.867172
11.0  0.867172
12.0  0.455572
Individual 2:
             1
2.0   1.000000
6.0   0.802728
8.0   0.802728
11.0  0.802728
12.0  0.297543
Individual 3:
             2
2.0   1.000000
6.0   0.924140
8.0   0.924140
11.0  0.924140
12.0  0.647131
Individual 4:
             3
2.0   1.000000
6.0   0.730078
8.0   0.730078
11.0  0.730078
12.0  0.176308
Individual 5:
             4
2.0   1.000000
6.0   0.653057
8.0   0.653057
11.0  0.653057
12.0  0.095317
Individual 6:
             5
2.0   1.000000
6.0   0.875674
8.0   0.875674
11.0  0.875674
12.0  0.480761


In [45]:
from lifelines.utils import concordance_index

# Fit the CoxPH model
cph.fit(df, duration_col="Follow_up_time", event_col="Event_indicator")

# Compute C-index
c_index = concordance_index(
    df["Follow_up_time"],  # Observed survival times
    -cph.predict_partial_hazard(df),  # Predicted risk scores (negative because higher risk = lower survival time)
    df["Event_indicator"]  # Event indicator (1 = event, 0 = censored)
)

print(f"C-index: {c_index:.4f}")


C-index: 1.0000


c:\Users\ASUS\pprojects\base\Lib\site-packages\lifelines\fitters\coxph_fitter.py:1589: ConvergenceWarning: The log-likelihood is getting suspiciously close to 0 and the delta is still large. There may be complete separation in the dataset. This may result in incorrect inference of coefficients. See https://stats.stackexchange.com/q/11109/11867 for more.

  warnings.warn(
c:\Users\ASUS\pprojects\base\Lib\site-packages\lifelines\fitters\coxph_fitter.py:1614: ConvergenceWarning: Newton-Raphson failed to converge sufficiently. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-model
  warnings.warn(


In [32]:
df

,Follow_up_time,Event_indicator,Embedding_1,Embedding_2,Embedding_3
0,12,1,2.500000e-01,1.5000,0.00005
1,8,0,2.000000e-21,0.0077,0.00100
2,15,1,1.000000e-18,0.1000,0.15000


In [50]:
df = load_regression_dataset()
df

,var1,var2,var3,T,E
0,0.595170,1.143472,1.571079,14.785652,1
1,0.209325,0.184677,0.356980,7.335846,1
2,0.693919,0.071893,0.557960,5.269797,1
3,0.443804,1.364646,0.374221,11.684092,1
4,1.613324,0.125566,1.921325,7.639492,1
...,...,...,...,...,...
195,0.137399,0.107748,0.354812,11.445457,1
196,0.637341,2.847188,1.459137,7.624627,1
197,1.109732,0.405561,0.018856,10.634620,1
198,0.031865,1.753759,0.252040,8.519718,1


In [57]:
from imblearn.over_sampling import SMOTE

# Applying SMOTE to balance the dataset
X = df[["var1", "var2", "var3", "T"]]
y = df["E"]

smote = SMOTE(random_state=123)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Combining the resampled data into a new DataFrame
resampled_df = pd.DataFrame(X_resampled, columns=["var1", "var2", "var3", "T"])
resampled_df["E"] = y_resampled

In [66]:
train_df

,var1,var2,var3,T,E
134,0.853077,0.221376,1.635539,9.780450,1
170,0.746388,0.267458,0.420036,11.942274,1
177,1.806666,3.535072,2.176759,5.810529,1
241,3.992947,0.303369,0.167932,12.086294,0
121,3.094217,0.764497,3.063756,7.644098,1
...,...,...,...,...,...
17,0.399475,0.822413,0.673405,6.433098,1
230,0.543304,2.035714,1.112076,8.941992,0
98,2.942882,0.163830,1.040333,7.988337,1
322,0.318062,0.077287,1.937679,4.728803,0


In [82]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(resampled_df, test_size=0.2, random_state=123)

train_tensor = torch.tensor(train_df.values, dtype=torch.float32)


import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split

In [83]:


train_df, val_df = train_test_split(resampled_df, test_size=0.2, random_state=123)
train_tensor = torch.tensor(train_df.values, dtype=torch.float32)


X = train_tensor[:, :3]  
T = train_tensor[:, 3]  
E = train_tensor[:, 4] 

# Define a simple neural network
class DeepSurvivalModel(nn.Module):
    def __init__(self, input_dim):
        super(DeepSurvivalModel, self).__init__()
        self.hidden = nn.Linear(input_dim, 16)
        self.output = nn.Linear(16, 1)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x, training=True):
        x = torch.relu(self.hidden(x))
        if training:
            x = self.dropout(x)
        return self.output(x)

# Cox partial likelihood loss
def cox_partial_likelihood(risk_scores, times, events):
    indices = torch.argsort(times, descending=True)
    risk_scores = risk_scores[indices]
    events = events[indices]

    log_risk = torch.cumsum(torch.exp(risk_scores), dim=0).log()
    log_likelihood = (risk_scores - log_risk) * events
    return -log_likelihood.sum()

# C-Index evaluation
def c_index(risk_scores, times, events):
    order = torch.argsort(times)
    risk_scores, events = risk_scores[order], events[order]
    n = len(times)
    concordant, permissible = 0, 0
    for i in range(n):
        for j in range(i + 1, n):
            if events[j] == 1:  # Only compare if j experienced an event
                permissible += 1
                if risk_scores[i] < risk_scores[j]:
                    concordant += 1
    return concordant / permissible if permissible > 0 else 0.0



model = DeepSurvivalModel(input_dim=3)
optimizer = optim.Adam(model.parameters(), lr=0.001)
early_stop_patience = 5
best_c_index = 0
no_improve_epochs = 0



n_epochs = 100
for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()
    risk_scores = model(X).squeeze()
    loss = cox_partial_likelihood(risk_scores, T, E)
    loss.backward()
    optimizer.step()


    model.eval()
    with torch.no_grad():
        risk_scores = model(X, training=False).squeeze()
        ci = c_index(risk_scores, T, E)

    print(f"epoch {epoch + 1}/{n_epochs}, Loss: {loss.item():.4f}, C-Index: {ci:.4f}")


    if ci > best_c_index:
        best_c_index = ci
        no_improve_epochs = 0
    else:
        no_improve_epochs += 1
        if no_improve_epochs >= early_stop_patience:
            print("early stopping")
            break


epoch 1/100, Loss: 741.7816, C-Index: 0.5742
epoch 2/100, Loss: 727.8021, C-Index: 0.5741
epoch 3/100, Loss: 749.5231, C-Index: 0.5736
epoch 4/100, Loss: 730.4083, C-Index: 0.5737
epoch 5/100, Loss: 727.5568, C-Index: 0.5736
epoch 6/100, Loss: 742.0102, C-Index: 0.5736
early stopping


In [89]:
val_tensor = torch.tensor(val_df.values, dtype=torch.float32)

X_val = val_tensor[:, :3]  
T_val = val_tensor[:, 3]   
E_val = val_tensor[:, 4]  

model.eval()
with torch.no_grad():
    risk_scores_val = model(X_val, training=False).squeeze()
    loss_val = cox_partial_likelihood(risk_scores_val, T_val, E_val)
    ci_val = c_index(risk_scores_val, T_val, E_val)

print(f"Validation Loss: {loss_val.item():.4f}, Validation C-Index: {ci_val:.4f}")


Validation Loss: 127.5884, Validation C-Index: 0.6207
